In [13]:
  import os

In [14]:
%pwd

'C:\\Users\\hp\\Desktop\\Sub\\Text_Summarizer_NLP'

In [15]:
os.chdir('C:\\Users\\hp\\Desktop\\Sub\\Text_Summarizer_NLP')


In [16]:
%pwd

'C:\\Users\\hp\\Desktop\\Sub\\Text_Summarizer_NLP'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [18]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            source_URL=config.source_URL,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [20]:
import os
import urllib.request as request
import zipfile
from src.textsummarizer.logging import logger
from src.textsummarizer.utils.common import get_size

In [21]:
class DataIngestion:
    
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(url = self.config.source_URL,filename = self.config.local_data_file)
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
            
            
    def extract_zip_file(self):
        """
        zip_file_path:str
        extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-09 10:34:45,103: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-09 10:34:45,106: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 10:34:45,108: INFO: common: Directory created at: artifacts]
[2024-07-09 10:34:45,108: INFO: common: Directory created at: artifacts/data_ingestion]
[2024-07-09 10:34:48,176: INFO: 2117429069: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FADC:D3BC1:2A44BA:39B866:668CC4EE
Accept-Ranges: bytes
Date: Tue, 09 Jul 2024 05:04:47 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100020-HYD
X-Cache: 